True

In [12]:
import json

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from google.cloud import pubsub_v1
import os
from dotenv import load_dotenv
import sys

load_dotenv()

import datetime
import json
import time

import tweepy
from google.cloud import pubsub_v1
from tweepy.streaming import StreamListener

publisher = pubsub_v1.PublisherClient()
topic_path = os.getenv('TOPIC_PATH')


# Load in a json file with your Tweepy API credentials
with open("./basic.json") as json_data:
    account_data = json.load(json_data)

# Select the account you want to listen with
auth = tweepy.OAuthHandler(os.getenv('CONSUMER_KEY'), os.getenv('CONSUMER_SECRET'))
auth.set_access_token(os.getenv('ACCESS_TOKEN'), os.getenv('TOKEN_SECRET'))

api = tweepy.API(auth, wait_on_rate_limit=True)

# Define the list of terms to listen to
lst_hashtags = ["#bitcoin"]

# Method to push messages to pubsub
def write_to_pubsub(data):
    try:
        if data["lang"] == "en": 
            test = json.dumps({
                "id": data["id"],
                "lang": data["lang"],
                "created_at": data["created_at"],
                "text": data["text"],
                "retweet_count": data["retweet_count"]
            }).encode("utf-8")
            print(test)
            publisher.publish(topic_path, data=test)
        
    except Exception as e:
        raise

# Method to format a tweet from tweepy
def reformat_tweet(tweet):
    x = tweet
    processed_doc = {
        "id": x["id"],
        "lang": x["lang"],
        "created_at": time.mktime(time.strptime(x["created_at"], "%a %b %d %H:%M:%S +0000 %Y")),
        "text": x["text"] if "favorite_count" in x else 0,
        "retweet_count": x["retweet_count"] if "retweet_count" in x else 0
    }
    return processed_doc

# Custom listener class
class StdOutListener(StreamListener):
    """ A listener handles tweets that are received from the stream.
    This is a basic listener that just pushes tweets to pubsub
    """

    def __init__(self):
        super(StdOutListener, self).__init__()
        self._counter = 0

    def on_status(self, data):
        write_to_pubsub(reformat_tweet(data._json))
        self._counter += 1
        return True

    def on_error(self, status):
        if status == 420:
            print("rate limit active")
            return False


# Start listening
l = StdOutListener()
stream = tweepy.Stream(auth, l, tweet_mode='extended')
stream.filter(track=lst_hashtags)

b'{"id": 1499886019725783040, "lang": "en", "created_at": 1646457268.0, "text": "The airdrop is very good and helpful it\'ll be more successful for the project .project in the feature and will cont\\u2026 https://t.co/708inzMt6B", "retweet_count": 0}'
b'{"id": 1499886022829563904, "lang": "en", "created_at": 1646457269.0, "text": "RT @chitaglorya__: giveaway 200$ | 2,800,000 IDR\\n\\n-- rt follow @flagnetwork_NFT \\n\\n-- rt &amp; quoted their pinned with \\n#FlagNetwork #IDO #cryp\\u2026", "retweet_count": 0}'
b'{"id": 1499886026625785856, "lang": "en", "created_at": 1646457269.0, "text": "RT @CmoBitcoin: The latest #bitcoin block 725895 with 2467 transactions was just mined by Luxor\\nTotal Fees:0.17548513\\nBlock Subsidy:6.25\\n#B\\u2026", "retweet_count": 0}'
b'{"id": 1499886027346825219, "lang": "en", "created_at": 1646457270.0, "text": "RT @AirdropStario: \\ud83d\\udca7 Two Monkey Airdrop \\ud83d\\udca7\\n\\n\\ud83c\\udfc6 Task:          \\u2795  Up to $3 worth of TMON\\n\\n\\ud8

KeyboardInterrupt: 